In [ ]:
from pprint import pprint as pp

import numpy as np, pandas as pd, holoviews as hv, geoviews as gv
import panel as pn
pn.extension()
hv.extension('bokeh')

In [ ]:
curve_opts = dict(
    framewise=True,
    xlim=(-10,10)
)
def show_func(frequency=1.0, amplitude=1.0, func_name='sin'):
    xs = np.linspace(-10,10,num=100)
    ys = amplitude * getattr(np, func_name)(frequency *xs)
    return hv.Curve( (xs,ys), 'x','y' ).opts(**curve_opts)
show_func(func_name='cos')

In [ ]:
dmap_func = (
    hv.DynamicMap(show_func, kdims=['frequency', 'amplitude', 'func_name'])
    .redim.range(frequency=(1., 5.),amplitude=(-2, 2.))
    .redim.values(func_name=['sin', 'cos', 'tan','log'])
)


In [ ]:
layout = pn.panel(dmap_func)

In [ ]:
layout

In [ ]:
layout.servable()

In [ ]:
type(layout)

In [ ]:
import param
from holoviews.streams import SingleTap

class Selection(param.Parameterized):
    typ = param.ObjectSelector(default='curve', objects=['curve','scatter'])
    
    def callback(self, x, y):
        x = 0 if x is None else x
        y = 0 if y is None else y
        el = hv.Curve if self.typ == 'curve' else hv.Scatter
        h = el(((x,0),(y,1)))
        return h.redim.range(x=(-1,1), y=(-1,1)).relabel('{},{}'.format(x,y)).options(show_grid=True)

    @param.depends('typ')
    def view(self):
        return hv.DynamicMap(self.callback, streams=[SingleTap(transient=True)])

selection     = Selection(name="Ouch")
pn.Column( 'Click on the plot or change plot type', selection.param, selection.view)